### Manual

In [1]:
import itertools
import pandas as pd

containers = [
    {'name': '10x', 'mult': 10, 'inh': 1},
    {'name': '17x', 'mult': 17, 'inh': 1},
    {'name': '20x', 'mult': 20, 'inh': 2},
    {'name': '31x', 'mult': 31, 'inh': 2},
    {'name': '37x', 'mult': 37, 'inh': 3},
    {'name': '50x', 'mult': 50, 'inh': 4},
    {'name': '73x', 'mult': 73, 'inh': 4},
    {'name': '80x', 'mult': 80, 'inh': 6},
    {'name': '89x', 'mult': 89, 'inh': 8},
    {'name': '90x', 'mult': 90, 'inh': 10},
]

combos1 = [(c,) for c in containers]
combos2 = list(itertools.combinations(containers, 2))

results = []

for combo in combos1 + combos2:
    v = sum(c['mult'] / c['inh'] for c in combo)
    fee = 50_000 if len(combo) == 2 else 0
    net = v - fee
    
    names = "+".join(c['name'] for c in combo)
    
    results.append({
        'Containers': names,
        'TotalValue': v,
        'Fee': fee,
        'NetValue': net,
    })

df = pd.DataFrame(results)
df = df.sort_values(by='NetValue', ascending=False)

print("BEST STRATEGY:")
print(df.head(10))

print("\nABSOLUTE BEST:")
print(df.iloc[0])


BEST STRATEGY:
  Containers  TotalValue  Fee   NetValue
6        73x   18.250000    0  18.250000
1        17x   17.000000    0  17.000000
3        31x   15.500000    0  15.500000
7        80x   13.333333    0  13.333333
5        50x   12.500000    0  12.500000
4        37x   12.333333    0  12.333333
8        89x   11.125000    0  11.125000
0        10x   10.000000    0  10.000000
2        20x   10.000000    0  10.000000
9        90x    9.000000    0   9.000000

ABSOLUTE BEST:
Containers      73x
TotalValue    18.25
Fee               0
NetValue      18.25
Name: 6, dtype: object


In [3]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# Container data
containers = [
    {'name': '10x', 'mult': 10, 'inh': 1},
    {'name': '17x', 'mult': 17, 'inh': 1},
    {'name': '20x', 'mult': 20, 'inh': 2},
    {'name': '31x', 'mult': 31, 'inh': 2},
    {'name': '37x', 'mult': 37, 'inh': 3},
    {'name': '50x', 'mult': 50, 'inh': 4},
    {'name': '73x', 'mult': 73, 'inh': 4},
    {'name': '80x', 'mult': 80, 'inh': 6},
    {'name': '89x', 'mult': 89, 'inh': 8},
    {'name': '90x', 'mult': 90, 'inh': 10},
]

container_names = [c['name'] for c in containers]

# Attractiveness = Multiplier / Inhabitants
weights = np.array([c['mult'] / c['inh'] for c in containers])
probs = weights / weights.sum()  # Normalize

combos1 = [(c,) for c in containers]
combos2 = list(itertools.combinations(containers, 2))

results = []

# Simulation for each combo
for combo in combos1 + combos2:
    name = "+".join(c['name'] for c in combo)
    value = sum(c['mult'] / c['inh'] for c in combo)
    fee = 50_000 if len(combo) == 2 else 0

    total_payout = []

    for _ in range(1000):
        picks = np.random.choice(container_names, size=7000, p=probs)

        # Count players per container
        count = {n: (picks == n).sum() for n in container_names}

        max_players = max(count[c['name']] for c in combo)

        payout = value / (max_players + 1) - fee
        total_payout.append(payout)

    avg_payout = np.mean(total_payout)
    worst_payout = np.min(total_payout)
    best_payout = np.max(total_payout)

    results.append({
        'Containers': name,
        'Value': value,
        'Fee': fee,
        'AveragePayout': avg_payout,
        'WorstPayout': worst_payout,
        'BestPayout': best_payout,
    })

df = pd.DataFrame(results).sort_values('AveragePayout', ascending=False)




In [6]:
import itertools
import pandas as pd
import numpy as np

# Settings
np.random.seed(0)
n_players = 7000
n_sims = 500
alphas = [0.2, 0.5, 1.0, 2.0]

# Container data
containers = [
    {'name': '10x', 'mult': 10, 'inh': 1},
    {'name': '17x', 'mult': 17, 'inh': 1},
    {'name': '20x', 'mult': 20, 'inh': 2},
    {'name': '31x', 'mult': 31, 'inh': 2},
    {'name': '37x', 'mult': 37, 'inh': 3},
    {'name': '50x', 'mult': 50, 'inh': 4},
    {'name': '73x', 'mult': 73, 'inh': 4},
    {'name': '80x', 'mult': 80, 'inh': 6},
    {'name': '89x', 'mult': 89, 'inh': 8},
    {'name': '90x', 'mult': 90, 'inh': 10},
]

# Precompute attractiveness A_i
for c in containers:
    c['A'] = c['mult'] / c['inh']

names = [c['name'] for c in containers]
A = np.array([c['A'] for c in containers])

results = []

for alpha in alphas:
    # softmax probabilities
    expA = np.exp(alpha * A)
    probs = expA / expA.sum()

    for c in containers:
        # single-container strategy
        value = c['A']
        fee = 0
        payouts = []
        for _ in range(n_sims):
            picks = np.random.choice(len(containers), size=n_players, p=probs)
            count = np.sum(picks == names.index(c['name']))
            payout = value / (count + 1) - fee
            payouts.append(payout)
        payouts = np.array(payouts)
        results.append({
            'Alpha': alpha,
            'Strategy': c['name'],
            'Mean': payouts.mean(),
            'VaR5%': np.percentile(payouts, 5),
            'Upside95%': np.percentile(payouts, 95),
        })

df = pd.DataFrame(results)
df


,Alpha,Strategy,Mean,VaR5%,Upside95%
0,0.2,10x,0.030791,0.028249,0.033670
1,0.2,17x,0.012902,0.012407,0.013460
2,0.2,20x,0.030731,0.028325,0.033557
3,0.2,31x,0.015875,0.015180,0.016685
4,0.2,37x,0.023760,0.022182,0.025485
5,0.2,50x,0.023339,0.021737,0.025050
6,0.2,73x,0.010764,0.010417,0.011148
7,0.2,80x,0.021072,0.019782,0.022447
8,0.2,89x,0.027274,0.025170,0.029513
9,0.2,90x,0.033774,0.030711,0.037045


In [19]:
import itertools
import numpy as np
import pandas as pd

# ─── PARAMETERS ──────────────────────────────────────────────────────────────
np.random.seed(42)
n_players = 7000
n_sims    = 1000
alphas    = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,2.0,3.0,4.0,5.0]   
# alphas    = [1.0,2.0,3.0,4.0,5.0] # herding intensities

# Container definitions
containers = [
    {'name':'10x', 'mult':10, 'inh':1},
    {'name':'17x', 'mult':17, 'inh':1},
    {'name':'20x', 'mult':20, 'inh':2},
    {'name':'31x', 'mult':31, 'inh':2},
    {'name':'37x', 'mult':37, 'inh':3},
    {'name':'50x', 'mult':50, 'inh':4},
    {'name':'73x', 'mult':73, 'inh':4},
    {'name':'80x', 'mult':80, 'inh':6},
    {'name':'89x', 'mult':89, 'inh':8},
    {'name':'90x', 'mult':90, 'inh':10},
]
FEE = 50_000

# Precompute V_i = multiplier/inhabitant
for c in containers:
    c['V'] = c['mult']/c['inh']

# Generate strategies
singletons = [(c,) for c in containers]
duos = list(itertools.combinations(containers, 2))
duos = sorted(duos, key=lambda combo: combo[0]['V']+combo[1]['V'], reverse=True)[:20]
strategies = singletons + duos

# ─── SIMULATION ──────────────────────────────────────────────────────────────
records = []
names = [c['name'] for c in containers]
V = np.array([c['V'] for c in containers])

for alpha in alphas:
    expV = np.exp(alpha * V)
    probs = expV / expV.sum()

    for strat in strategies:
        strat_names = "+".join(c['name'] for c in strat)
        strat_value = sum(c['V'] for c in strat)
        strat_fee   = FEE if len(strat)==2 else 0

        payouts = np.zeros(n_sims)
        for sim in range(n_sims):
            picks = np.random.choice(len(containers), size=n_players, p=probs)
            count = sum((picks == names.index(c['name'])).sum() for c in strat)
            payouts[sim] = strat_value/(count+1) - strat_fee

        mu    = payouts.mean()
        var5  = np.percentile(payouts, 5)
        up95  = np.percentile(payouts, 95)
        sigma = payouts.std(ddof=1)
        score = mu/sigma if sigma>0 else np.nan

        records.append({
            'Alpha': alpha,
            'Strategy': strat_names,
            'Mean': mu,
            'VaR5%': var5,
            'Upside95%': up95,
            'Score': score,
        })

df = pd.DataFrame(records)

# ─── SELECT BEST FOR EACH ALPHA ───────────────────────────────────────────────
best_per_alpha = []
for alpha in alphas:
    sub = df[df['Alpha']==alpha]
    # Choose by highest Score; tie‑breaker: highest Mean
    best = sub.sort_values(['Score','Mean'], ascending=[False,False]).iloc[0]
    best_per_alpha.append(best)

best_df = pd.DataFrame(best_per_alpha)
print("=== BEST STRATEGIES BY ALPHA ===")
print(best_df[['Alpha','Strategy','Mean','VaR5%','Upside95%','Score']].to_string(index=False))


=== BEST STRATEGIES BY ALPHA ===
 Alpha Strategy      Mean     VaR5%  Upside95%        Score
   0.1      73x  0.015971  0.015271   0.016682 3.676582e+01
   0.2      73x  0.010778  0.010411   0.011148 4.699955e+01
   0.3      73x  0.007955  0.007743   0.008173 6.002277e+01
   0.4      73x  0.006333  0.006186   0.006479 6.981949e+01
   0.5      73x  0.005348  0.005244   0.005459 8.201913e+01
   0.6      73x  0.004715  0.004631   0.004800 9.230415e+01
   0.7      73x  0.004285  0.004217   0.004355 1.024799e+02
   0.8      73x  0.003971  0.003916   0.004030 1.168493e+02
   0.9      73x  0.003738  0.003693   0.003784 1.344663e+02
   1.0      73x  0.003558  0.003518   0.003600 1.397888e+02
   2.0      73x  0.002832  0.002816   0.002847 2.912815e+02
   3.0      37x 12.333333 12.333333  12.333333 6.939577e+15
   4.0      80x 13.333333 13.333333  13.333333 7.502245e+15
   5.0      80x 13.333333 13.333333  13.333333 7.502245e+15


In [20]:
# ─── after your existing simulation and best_df creation ────────────────────────

# Count how often each strategy wins across alphas
frequency = best_df['Strategy'].value_counts().reset_index()
frequency.columns = ['Strategy', 'Frequency']

# Merge in average Mean payout across alphas for tie‑breaking
mean_stats = df.groupby('Strategy')['Mean'].mean().reset_index()
mean_stats.columns = ['Strategy', 'AvgMean']

freq_df = frequency.merge(mean_stats, on='Strategy')

# Sort by Frequency desc, then AvgMean desc
freq_df = freq_df.sort_values(['Frequency','AvgMean'], ascending=[False,False])

print("=== STRATEGY FREQUENCY ACROSS ALPHAS ===")
print(freq_df.to_string(index=False))

# Final pick = top row
final = freq_df.iloc[0]
print(f"\n>>> FINAL RECOMMENDATION: {final['Strategy']}  "
      f"(won {int(final['Frequency'])}/{len(alphas)} alphas, "
      f"avg mean = {final['AvgMean']:.6f})")

=== STRATEGY FREQUENCY ACROSS ALPHAS ===
Strategy  Frequency  AvgMean
     73x         11 0.005528
     80x          2 3.743205
     37x          1 3.665090

>>> FINAL RECOMMENDATION: 73x  (won 11/14 alphas, avg mean = 0.005528)


In [22]:
from tqdm import trange
import random

containers = [
    {'name': '10x', 'mult': 10, 'threshold': 1},
    {'name': '17x', 'mult': 17, 'threshold': 1},
    {'name': '20x', 'mult': 20, 'threshold': 2},
    {'name': '31x', 'mult': 31, 'threshold': 2},
    {'name': '37x', 'mult': 37, 'threshold': 3},
    {'name': '50x', 'mult': 50, 'threshold': 4},
    {'name': '73x', 'mult': 73, 'threshold': 4},
    {'name': '80x', 'mult': 80, 'threshold': 6},
    {'name': '89x', 'mult': 89, 'threshold': 8},
    {'name': '90x', 'mult': 90, 'threshold': 10},
]

NUM_CONTAINERS = len(containers)
NUM_AGENTS = 7000
FEE = 50000

# Each agent's strategy is a list of container indices, length 1 or 2
agents = []
for _ in range(NUM_AGENTS):
    # Random single or pair
    picks = random.sample(range(NUM_CONTAINERS), random.choice([1, 2]))
    agents.append(picks)

# Build the initial picks_count
picks_count = [0]*NUM_CONTAINERS
for strat in agents:
    for c in strat:
        picks_count[c] += 1

def payoff_for_agent(strat, picks_count):
    """Compute payoff for a single agent with the given strategy,
       given the current picks_count (which includes that agent's picks)."""
    total_payoff = 0.0
    for c_idx in strat:
        c_info = containers[c_idx]
        count = picks_count[c_idx]
        # If threshold not met, that container pays 0
        if count >= c_info['threshold']:
            container_value = c_info['mult'] * 100000
            total_payoff += container_value / count
        else:
            total_payoff += 0
    # Subtract fee if picking 2
    if len(strat) == 2:
        total_payoff -= FEE
    return total_payoff

def payoff_if_switch(agent_old_strat, agent_new_strat, picks_count):
    """
    Compute payoff if an agent switches from old_strat to new_strat.
    We'll do a partial update on picks_count, then revert.
    """
    # Temporarily remove old picks
    for c in agent_old_strat:
        picks_count[c] -= 1

    # Add new picks
    for c in agent_new_strat:
        picks_count[c] += 1

    # Calculate payoff
    new_pay = payoff_for_agent(agent_new_strat, picks_count)

    # revert picks_count
    for c in agent_new_strat:
        picks_count[c] -= 1
    for c in agent_old_strat:
        picks_count[c] += 1

    return new_pay

possible_strategies = []
# Single-container strategies
for i in range(NUM_CONTAINERS):
    possible_strategies.append([i])
# Pair strategies
for i in range(NUM_CONTAINERS):
    for j in range(i+1, NUM_CONTAINERS):
        possible_strategies.append([i, j])

NUM_ITERATIONS = 5  # or however many you want

for iteration in trange(NUM_ITERATIONS, desc="Main Iteration"):
    # We'll do one pass over all agents
    for i in range(NUM_AGENTS):
        old_strat = agents[i]
        old_pay = payoff_for_agent(old_strat, picks_count)

        best_pay = old_pay
        best_strat = old_strat

        # Evaluate switching to each possible strategy
        for strat in possible_strategies:
            new_pay = payoff_if_switch(old_strat, strat, picks_count)
            if new_pay > best_pay:
                best_pay = new_pay
                best_strat = strat

        # If we found a better strategy, update it
        if best_strat != old_strat:
            # remove old picks
            for c in old_strat:
                picks_count[c] -= 1
            # add new picks
            for c in best_strat:
                picks_count[c] += 1
            # store final
            agents[i] = best_strat

    # End of iteration
    # Optionally measure how many switches happened or total payoff


Main Iteration: 100%|██████████| 5/5 [00:00<00:00,  5.15it/s]


In [23]:
# 1. Count how many people picked each container
final_picks_count = [0]*NUM_CONTAINERS
for strat in agents:
    for c in strat:
        final_picks_count[c] += 1

# 2. Compute the payoff for each container
container_payoff = [0.0]*NUM_CONTAINERS
for i, c_info in enumerate(containers):
    total_value = c_info['mult'] * 100000  # e.g. 73 => 730k
    if final_picks_count[i] >= c_info['threshold']:
        container_payoff[i] = total_value / final_picks_count[i]
    else:
        container_payoff[i] = 0.0

# 3. Print container-level info
print("Final container stats:")
for i, c_info in enumerate(containers):
    print(
        f"Container {c_info['name']} | Picks: {final_picks_count[i]:4d} "
        f"| Threshold: {c_info['threshold']:2d} "
        f"| Final payoff each: {container_payoff[i]:,.0f}"
    )


Final container stats:
Container 10x | Picks:  140 | Threshold:  1 | Final payoff each: 7,143
Container 17x | Picks:  239 | Threshold:  1 | Final payoff each: 7,113
Container 20x | Picks:  281 | Threshold:  2 | Final payoff each: 7,117
Container 31x | Picks:  437 | Threshold:  2 | Final payoff each: 7,094
Container 37x | Picks:  521 | Threshold:  3 | Final payoff each: 7,102
Container 50x | Picks:  704 | Threshold:  4 | Final payoff each: 7,102
Container 73x | Picks: 1029 | Threshold:  4 | Final payoff each: 7,094
Container 80x | Picks: 1127 | Threshold:  6 | Final payoff each: 7,098
Container 89x | Picks: 1254 | Threshold:  8 | Final payoff each: 7,097
Container 90x | Picks: 1268 | Threshold: 10 | Final payoff each: 7,098
